In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku
import pandas as pd
import string

Using TensorFlow backend.


In [7]:
lagu = pd.read_csv("/content/drive/My Drive/Colab Notebooks/song_df_indo.csv")
lagu.head()

,Unnamed: 0,id,judul,penyanyi,link,lirik,lang
0,8,9,A Lotta Love,3 Diva,https://lirik.kapanlagi.com/artis/3-diva/a-lot...,"(*) Iâ¦ I, I, I, Iâ¦ Need A Lotta Love Youâ...",id
1,10,11,A Minor,Kaneshiba,https://lirik.kapanlagi.com/artis/kaneshiba/a-...,Dari A minor Pindahnya ke D minor Masuk kunci ...,id
2,11,12,A N G,Naff,https://lirik.kapanlagi.com/artis/naff/a-n-g/,NAFF - A.N.G Seluruh hati tlah kudatangi Hanya...,id
3,17,18,A S A,Arwana,https://lirik.kapanlagi.com/artis/arwana/a-s-a/,Dalam malam kucoba mengerti Tentang luka yang ...,id
4,20,21,A Tribute To Maia,Tha Law,https://lirik.kapanlagi.com/artis/tha-law/a-tr...,Now throw ur hand in the air buat duo Maia Jar...,id


In [8]:
lagu = lagu[lagu.penyanyi=="Didi Kempot"]
lagu.head()

,Unnamed: 0,id,judul,penyanyi,link,lirik,lang
150,205,206,Aduh Mana Tahan,Didi Kempot,https://lirik.kapanlagi.com/artis/didi-kempot/...,"Aduh aduh aduh ruminten, percaya pada diriku, ...",id
757,1000,1001,Ambyar,Didi Kempot,https://lirik.kapanlagi.com/artis/didi-kempot/...,Wis kebacut ambyar remuk sing ning ati Opo nge...,id
878,1157,1158,Anggar Bini,Didi Kempot,https://lirik.kapanlagi.com/artis/didi-kempot/...,"Kudu lilo, yen to aku anglemboro Panyuwunku pu...",id
1196,1584,1585,Awu Merapi,Didi Kempot,https://lirik.kapanlagi.com/artis/didi-kempot/...,Neng ngendi kudu lunga Nggoleki kenya sing tak...,id
1400,1852,1853,Bangjo Malioboro,Didi Kempot,https://lirik.kapanlagi.com/artis/didi-kempot/...,Lampu bangjo neng prapatan malioboro.. nganti ...,id


In [9]:
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    return txt.lower()

corpus = [clean_text(x) for x in list(lagu["lirik"])]
corpus[0]

'aduh aduh aduh ruminten percaya pada diriku sumpah mati ku cinta kamu bulan depan aku melamarmu gak lucu abang komar gak salah tu emangnya gampang nglamar kerja dulu dong baru nglamar aduh duh kamu hai abang komar apa gak salah akan mau melamar hanya sama minten ai ai kucinta berat kalau minten tolak aduh duh bisa sekarat aduh bang komar pemuda tampan kerjalah dulu kalau mau pacaran percayalah minten abang tunggu panggilan gajinya selangit aduh duh banyak obyekan aduh aduh aduh bang komar sadarlah jangan menghayal banyak sarjana pengangguran nunggu panggilan sampai ubanan aduh ruminten gadis pujaan apa ruminten cari jutawan bukanya ruminten ay ay mata duitan yang penting abang aduh duh punya kerjaan jikalau cinta telah menyatu pahit empedu terasa madu marilah kita ay ay saling mengikat aku tak mau aduh duh si cinta kilat aduh bang komar kusayang kamu aduh ruminten kucinta kamu'

In [10]:
len(corpus)

76

In [0]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

In [15]:
inp_sequences, total_words = get_sequence_of_tokens(corpus)
#print(inp_sequences[:10])

print(total_words)

from keras.preprocessing.sequence import pad_sequences
import keras.utils as ku

2093


In [16]:
import numpy as np
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

print(max_sequence_len)

252


In [19]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    
    #tambahan
    model.add(Dense(100, activation='relu'))
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_words)
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 251, 10)           20930     
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 2093)              211393    
Total params: 286,823
Trainable params: 286,823
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
model.fit(predictors, label, epochs=100, batch_size=1024)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
8951/8951 [==============================] - 33s 4ms/step - loss: 7.6429
Epoch 2/100
8951/8951 [==============================] - 32s 4ms/step - loss: 7.5376
Epoch 3/100
8951/8951 [==============================] - 32s 4ms/step - loss: 6.9252
Epoch 4/100
8951/8951 [==============================] - 32s 4ms/step - loss: 6.7526
Epoch 5/100
8951/8951 [==============================] - 32s 4ms/step - loss: 6.7141
Epoch 6/100
8951/8951 [==============================] - 32s 4ms/step - loss: 6.6987
Epoch 7/100
8951/8951 [==============================] - 35s 4ms/step - loss: 6.6910
Epoch 8/100
8951/8951 [==============================] - 31s 3ms/step - loss: 6.6892
Epoch 9/100
8951/8951 [==============================] - 31s 3ms/step - loss: 6.6852
Epoch 10/100
8951/8951 [==============================] - 31s 3ms/step - loss: 6.6821
Epoch 11/100
8951/8951 [==============================] - 31s 3ms/step - loss: 6.6779
Epoch 12/100
8951/8951 [==============================] - 31s 3

In [0]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [33]:
print (generate_text("kowe ayu tenan dek", 25, model, max_sequence_len))

Kowe Ayu Tenan Dek Kere Kowe Kere Piye Piye Piye Piye Piye Piye Piye Piye Piye Piye Piye Piye Piye Piye Piye Piye Piye Piye Piye Piye Piye Piye


In [0]:
from pickle import dump
model.save('model_lagu_dikem_100_epoch.h5')
#save the tokenizer
dump(tokenizer, open('tokenizer_lagu_dikem_100_epoch.pkl', 'wb'))

In [0]:
#tes 1000 epoch
model2 = create_model(max_sequence_len, total_words)
print(model2.summary())
model2.fit(predictors, label, epochs=1000, batch_size=1024)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 251, 10)           20930     
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_6 (Dense)              (None, 2093)              211393    
Total params: 286,823
Trainable params: 286,823
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1000
8951/8951 [==============================] - 35s 4ms/step - loss: 7.6422
Epoch 2/1000
8951/8951 [==============================] - 34s 4ms/step - loss: 7.5216
Epoch 3/1000
8951/8951 [==============================] - 34s 4ms/step - loss: 6.9374
Epoch 4/1000
8951/8951 [==============================] - 34s 4ms/step - loss: 6.7541
Epoch 5/1000
8951/8951 [==============================] - 33s 4ms/step - loss: 6.7088
Epoch 6/1000
8951/8951 [==============================] - 33s 4ms/step - loss: 6.6986
Epoch 7/1000
8951/8951 [==============================] - 33s 4ms/step - loss: 6.6903
Epoch 8/1000
8951/8951 [==============================] - 33s 4ms/step - loss: 6.6879
Epoch 9/1000
8951/8951 [==============================] - 33s 4ms/step - loss: 6.6852
Epoch 10/1000
8951/8951 [==============================] - 33s 4ms/step - loss: 6.6816
Epoch 11/1000
8951/8951 [==============================] - 34s 4ms/step - loss: 6.6778
Epoch 12/1000
8951/8951 [===========================